<a href="https://colab.research.google.com/github/kushc2004/LLM/blob/main/Manager.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 4.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [7]:
!git clone https://github.com/kushc2004/LLM.git

fatal: destination path 'LLM' already exists and is not an empty directory.


In [4]:
# AGENT
from typing import Dict, Optional, List, Tuple
from transformers import AutoModelForCausalLM, AutoTokenizer
from accelerate import init_empty_weights, infer_auto_device_map, dispatch_model
import torch

class Agent:
    def __init__(self, name, description, model_name, **kwargs):
        self.name = name
        self.description = description
        self.system_prompt = "You're a helpful assistant."
        self.kwargs = kwargs
        self.model_name = model_name

        # Load the tokenizer
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)

        # Initialize the model with empty weights and use `init_empty_weights`
        with init_empty_weights():
            self.model = AutoModelForCausalLM.from_pretrained(self.model_name, low_cpu_mem_usage=True)

        # Infer the device map
        device_map = infer_auto_device_map(self.model, max_memory={0: "12GiB", "cpu": "30GiB"})

        # Dispatch the model to the appropriate devices
        self.model = dispatch_model(self.model, device_map=device_map)

    def llm_call(
        self,
        prompt: Optional[str] = None,
        messages: Optional[List] = None,
        seed: int = 10,
    ) -> str:
        # Ensure exactly one of prompt or messages is provided
        assert (prompt is None) != (messages is None)

        # Ensure if messages is provided, it is a list of dicts with role and content
        if messages is not None:
            assert isinstance(messages, list)
            for message in messages:
                assert isinstance(message, dict)
                assert "role" in message
                assert "content" in message

        if prompt is not None:
            messages = [
                {"role": "system", "content": self.system_prompt},
                {"role": "user", "content": prompt},
            ]

        # Concatenate messages into a single prompt
        full_prompt = "\n".join([f"{msg['role']}: {msg['content']}" for msg in messages])

        # Tokenize the input
        inputs = self.tokenizer(full_prompt, return_tensors="pt")

        # Generate response
        with torch.no_grad():
            outputs = self.model.generate(**inputs, max_length=200, do_sample=True, top_p=0.95, top_k=50, temperature=0.7)

        # Decode the response
        response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)

        return response

    def generate_reply(
        self,
        task: str,
        state: Dict,
        sender: "Agent1",
    ) -> Tuple[str, Dict]:
        return (
            "This is a reply from the agent. REPLY NOT IMPLEMENTED! Terminate the whole process!",
            state,
        )



In [5]:
# MANAGER
from typing import Dict, Tuple
import json
from transformers import LlamaForCausalLM, LlamaTokenizer
import torch

class GroupChatManager(Agent):
    def __init__(
        self, model_name: str, agents: List[Agent], max_rounds: int = 12, **kwargs
    ):
        super().__init__(
            name="GroupChatManager",
            description="This is a manager agent that chooses which agent to work on the problem next and organizes the conversation within its team.",
            **kwargs,
        )

        self.agents = agents
        self.conversation_state = {
            "round": 0,
        }
        self.max_rounds = max_rounds
        self.history = []
        self.prompt_template = """

You're a manager in a team of optimization experts. The goal of the team is to solve an optimization problem. Your task is to choose the next expert to work on the problem based on the current situation.
- The user has already given us the problem description, the objective function, and the parameters. Only call the user proxy if there is a problem or something ambiguous or missing.

Here's the list of agents in your team:
-----
{agents}
-----

And here's the history of the conversation so far:
-----
{history}
-----


Considering the history, if you think the problem is solved, type DONE. Otherwise, generate a json file with the following format:
{{
    "agent_name": "Name of the agent you want to call next",
    "task": "The task you want the agent to carry out"
}}

to identify the next agent to work on the problem, and also the task it has to carry out.
- If there is a runtime error, ask the the programmer agent to fix it.
- Only generate the json file, and don't generate any other text.
- If the latest message in history says that the code is fixed, ask the evaluator agent to evaluate the code!

"""

        # Load the Llama model and tokenizer
        self.tokenizer = LlamaTokenizer.from_pretrained(model_name)
        self.model = LlamaForCausalLM.from_pretrained(model_name)

    def llm_call(self, prompt: str, seed: int) -> str:
        inputs = self.tokenizer(prompt, return_tensors="pt")
        outputs = self.model.generate(**inputs, max_length=512)
        response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        return response

    def solve(self, state: Dict) -> Tuple[str, Dict]:
        self.history = []

        while True:
            if self.conversation_state["round"] >= self.max_rounds:
                return "The problem is not solved.", state

            print("=" * 20)
            print("=" * 20)
            print("Round", self.conversation_state["round"])

            agents_list = "".join(
                [
                    "-" + agent.name + ": " + agent.description + "\n"
                    for agent in self.agents
                ]
            )

            prompt = self.prompt_template.format(
                agents=agents_list,
                history="\n".join([json.dumps(item[0]) for item in self.history]),
            )

            cnt = 3
            while True and cnt > 0:
                try:
                    response = self.llm_call(prompt=prompt, seed=cnt)

                    decision = response.strip()
                    if "```json" in decision:
                        decision = decision.split("```json")[1].split("```")[0]
                    decision = decision.replace("\\", "")

                    if decision == "DONE":
                        print("DONE")
                        return "The problem is solved.", state
                    decision = json.loads(decision)
                    break

                except Exception as e:
                    print(response)
                    print(e)
                    cnt -= 1

                    print("Invalid decision. Trying again ...")
                    if cnt == 0:
                        import traceback

                        err = traceback.format_exc()
                        print(err)

            print(
                "---- History:\n",
                "\n".join([json.dumps(item[0]) for item in self.history]),
            )

            print(f"\n---- Decision:||{decision}||\n")

            if not decision["agent_name"] in [agent.name for agent in self.agents]:
                raise ValueError(
                    f"Decision {decision} is not a valid agent name. Please choose from {self.agents}"
                )
            else:
                agent = [
                    agent
                    for agent in self.agents
                    if agent.name == decision["agent_name"]
                ][0]

                message, new_state = agent.generate_reply(
                    task=decision["task"],
                    state=state,
                    sender=self,
                )

                with open(
                    f"{state['log_folder']}/log_{self.conversation_state['round']}.json",
                    "w",
                ) as f:
                    json.dump(state, f, indent=4)

                state = new_state

                decision["result"] = message
                self.history.append((decision, state))

                with open(state["log_folder"] + "/selection_log.json", "w") as f:
                    json.dump([d for (d, s) in self.history], f, indent=4)

                if "code" in state:
                    with open(state["log_folder"] + "/code.py", "w") as f:
                        f.write(state["code"])

                self.conversation_state["round"] += 1

In [29]:
# MANAGER EXAMPLE
# First, let's define the necessary classes from agent.py and manager.py
from typing import Dict, Optional, List
from unittest.mock import MagicMock
import unittest
import json

class Agent:
    def __init__(self, name, description, client, llm="gpt-3.5-turbo", **kwargs):
        self.name = name
        self.description = description
        self.client = client
        self.system_prompt = "You're a helpful assistant."
        self.kwargs = kwargs
        self.llm = llm

    def llm_call(
        self,
        prompt: Optional[str] = None,
        messages: Optional[List] = None,
        seed: int = 10,
    ) -> str:
        model = self.llm
        # make sure exactly one of prompt or messages is provided
        assert (prompt is None) != (messages is None)
        # make sure if messages is provided, it is a list of dicts with role and content
        if messages is not None:
            assert isinstance(messages, list)
            for message in messages:
                assert isinstance(message, dict)
                assert "role" in message
                assert "content" in message

        if not prompt is None:
            messages = [
                {"role": "system", "content": self.system_prompt},
                {"role": "user", "content": prompt},
            ]

        # Mocking the response for testing purposes
        content = "Mocked response content"
        return content

    def generate_reply(
        self,
        task: str,
        state: Dict,
        sender: "Agent",
    ) -> Tuple[str, Dict]:
        return (
            "This is a reply from the agent. REPLY NOT IMPLEMENTED! Terminate the whole process!",
            state,
        )

class GroupChatManager(Agent):
    def __init__(
        self, client, agents: List[Agent], max_rounds: int = 12, **kwargs
    ):
        super().__init__(
            name="GroupChatManager",
            description="This is a manager agent that chooses which agent to work on the problem next and organizes the conversation within its team.",
            client=client,
            **kwargs,
        )

        self.agents = agents
        self.conversation_state = {
            "round": 0,
        }
        self.max_rounds = max_rounds
        self.history = []
        self.prompt_template = """

You're a manager in a team of optimization experts. The goal of the team is to solve an optimization problem. Your task is to choose the next expert to work on the problem based on the current situation.
- The user has already given us the problem description, the objective function, and the parameters. Only call the user proxy if there is a problem or something ambiguous or missing.

Here's the list of agents in your team:
-----
{agents}
-----

And here's the history of the conversation so far:
-----
{history}
-----


Considering the history, if you think the problem is solved, type DONE. Otherwise, generate a json file with the following format:
{{
    "agent_name": "Name of the agent you want to call next",
    "task": "The task you want the agent to carry out"
}}

to identify the next agent to work on the problem, and also the task it has to carry out.
- If there is a runtime error, ask the the programmer agent to fix it.
- Only generate the json file, and don't generate any other text.
- If the latest message in history says that the code is fixed, ask the evaluator agent to evaluate the code!

"""

    def solve(self, state: Dict) -> Tuple[str, Dict]:
        self.history = []

        while True:
            if self.conversation_state["round"] >= self.max_rounds:
                return "The problem is not solved.", state

            print("=" * 20)
            print("=" * 20)
            print("Round", self.conversation_state["round"])
            # print(json.dumps(state, indent=4))

            agents_list = "".join(
                [
                    "-" + agent.name + ": " + agent.description + "\n"
                    for agent in self.agents
                ]
            )

            prompt = self.prompt_template.format(
                agents=agents_list,
                history="\n".join([json.dumps(item[0]) for item in self.history]),
            )

            cnt = 3
            while True and cnt > 0:
                try:
                    response = self.llm_call(prompt=prompt, seed=cnt)

                    decision = response.strip()
                    if "```json" in decision:
                        decision = decision.split("```json")[1].split("```")[0]
                    decision = decision.replace("\\", "")

                    if decision == "DONE":
                        print("DONE")
                        return "The problem is solved.", state
                    decision = json.loads(decision)
                    break

                except Exception as e:
                    print(response)
                    print(e)
                    cnt -= 1

                    print("Invalid decision. Trying again ...")
                    if cnt == 0:
                        import traceback

                        err = traceback.format_exc()
                        print(err)

            print(
                "---- History:\n",
                "\n".join([json.dumps(item[0]) for item in self.history]),
            )

            print(f"\n---- Decision:||{decision}||\n")

            # wait for the user to press enter
            # input()

            if not decision["agent_name"] in [agent.name for agent in self.agents]:
                raise ValueError(
                    f"Decision {decision} is not a valid agent name. Please choose from {self.agents}"
                )
            else:
                agent = [
                    agent
                    for agent in self.agents
                    if agent.name == decision["agent_name"]
                ][0]

                message, new_state = agent.generate_reply(
                    task=decision["task"],
                    state=state,
                    sender=self,
                )

                with open(
                    f"/log_{self.conversation_state['round']}.json",
                    "a+",
                ) as f:
                    json.dump(state, f, indent=4)

                state = new_state

                decision["result"] = message
                self.history.append((decision, state))

                with open( "/selection_log.json", "a+") as f:
                    json.dump([d for (d, s) in self.history], f, indent=4)

                if "code" in state:
                    with open("/code.py", "a+") as f:
                        f.write(state["code"])

                self.conversation_state["round"] += 1

# Now, let's define the test case

class TestGroupChatManager(unittest.TestCase):
    def setUp(self):
        # Create a mock client
        mock_client = MagicMock()

        # Create mock agents
        self.formulator = Agent(name="Formulator", description="Formulates problems", client=mock_client)
        self.programmer = Agent(name="Programmer", description="Writes code", client=mock_client)
        self.evaluator = Agent(name="Evaluator", description="Evaluates solutions", client=mock_client)

        # Initialize GroupChatManager with mock agents
        self.manager = GroupChatManager(
            client=mock_client,
            agents=[self.formulator, self.programmer, self.evaluator],
            max_rounds=3
        )

        # Mock state
        self.state = {
            "background": "Test background",
            "problem_type": "LP",
            "parameters": [],
            "constraint": [],
            "variables": [],
            "objective": [],
            "solution_status": None,
            "solver_output_status": None,
            "error_message": None,
            "obj_val": None,
            "log_folder": "logs/test_log/",
            "data_json_path": "data/test_data.json",
        }

        def test_solve(self):
            # Mock the llm_call method to return a sequence of valid decisions
            self.manager.llm_call = MagicMock(side_effect=[
                '{"agent_name": "Formulator", "task": "Formulate the problem"}',
                '{"agent_name": "Programmer", "task": "Write the code"}',
                '{"agent_name": "Evaluator", "task": "Evaluate the solution"}',
                'DONE'
            ])

            # Run the solve method
            result, state = self.manager.solve(self.state)

            # Check the result
            self.assertEqual(result, "The problem is solved.")
            self.assertEqual(self.manager.conversation_state["round"], 4)
            self.formulator.generate_reply.assert_called_once()
            self.programmer.generate_reply.assert_called_once()
            self.evaluator.generate_reply.assert_called_once()
            self.assertEqual(state, self.state)

if __name__ == "__main__":
    unittest.main(argv=[''], verbosity=2, exit=False)


----------------------------------------------------------------------
Ran 0 tests in 0.000s

OK


In [18]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.3 MB/s eta 0:00:00


In [22]:
# FORMULATOR
from typing import Dict
import json
from transformers import pipeline

fix_prompt_template = """
You are a mathematical formulator working with a team of optimization experts. The objective is to tackle a complex optimization problem, and your role is to fix a previously modelled {target}.

Recall that the {target} you modelled was

-----
{constraint}
-----

and your formulation you provided was

-----
{formulation}
-----

The error message is

-----
{error}
-----

Here are the variables you have so far defined:

-----
{variables}
-----

Here are the parameters of the problem

-----
{parameters}
-----

Your task is carefully inspect the old {target} and fix it when you find it actually wrong.
After fixing it modify the formulation. Please return the fixed JSON string for the formulation.

The current JSON is

-----
{json}
-----

"""

prompt_template = """
You are an expert mathematical formulator and an optimization professor at a top university. Your task is to model {targetType} of the problem in the standard LP or MILP form.

Here is a {targetType} we need you to model:
-----
{targetDescription}
-----

Here is some context on the problem:
-----
{background}
-----

Here is the list of available variables:
-----
{variables}
-----

And finally, here is list of input parameters:
-----
{parameters}
-----

First, take a deep breath and explain how we should define the {targetType}. Feel free to define new variables if you think it is necessary. Then, generate a json file accordingly with the following format (STICK TO THIS FORMAT!):


{{
    "{targetType}": {{
      "description": "The description of the {targetType}",
      "formulation": "The LaTeX mathematical expression representing the formulation of the {targetType}"
    }},
    "auxiliary_constraints": [
        {{
            "description": "The description of the auxiliary constraint",
            "formulation": "The LaTeX mathematical expression representing the formulation of the auxiliary constraint"
        }}
    ]
    "new_variables": [
        {{
            "definition": "The definition of the variable",
            "symbol": "The symbol for the variable",
            "shape": [ "symbol1", "symbol2", ... ]
        }}
    ],

}}

- Your formulation should be in LaTeX mathematical format (do not include the $ symbols).
- Note that I'm going to use python json.loads() function to parse the json file, so please make sure the format is correct (don't add ',' before enclosing '}}' or ']' characters.
- Generate the complete json file and don't omit anything.
- Use '```json' and '```' to enclose the json file.
- Important: You can not define new parameters. You can only define new variables.Use CamelCase and full words for new variable symbols, and do not include indices in the symbol (e.g. ItemsSold instead of itemsSold or items_sold or ItemsSold_i)
- Use \\textup{{}} when writing variable and parameter names. For example (\\sum_{{i=1}}^{{N}} \\textup{{ItemsSold}}_{{i}} instead of \\sum_{{i=1}}^{{N}} ItemsSold_{{i}})
- Use \\quad for spaces.
- Use empty list ([]) if no new variables are defined.
- Always use non-strict inequalities (e.g. \\leq instead of <), even if the constraint is strict.
- Define auxiliary constraints when necessary. Set it to an empty list ([]) if no auxiliary constraints are needed. If new auxiliary constraints need new variables, add them to the "new_variables" list too.

Take a deep breath and solve the problem step by step.

"""

class Formulator(Agent):
    def __init__(self, client=None, **kwargs):
        super().__init__(
            name="Formulator",
            description="This is a mathematical formulator agent that is an expert in mathematical and optimization modeling and can define and modify variables, constraints, and objective functions. Does 3 things: 1) Defining constraints, variables, and objective functions, 2) Modifying constraints, variables, and objective functions, 3) Other things related to mathematical formulation. If the history is empty, start from this agent.",
            client=client,
            **kwargs,
        )
        self.llm = pipeline('text-generation', model='huggingface/llama')

    def _formulate(self, target_type: str, state):
        for target in state[target_type]:
            if target["status"] == "not_formulated":
                self._create_new_formulation(target, target_type, state)
            elif target["status"] == "runtime_error":
                self._fix_available_formulation(target, target_type, state)
            elif target["status"] == "formulated":
                continue
            else:
                error_msg = f"Invalid status: {json.dumps(target, indent=4)}"
                raise RuntimeError(error_msg)

        return

    def _create_new_formulation(self, target, target_type: str, state):
        print(f"Formulating {target_type} ...")
        context = {}
        context[target_type] = {}
        prompt = prompt_template.format(
            background=state["background"],
            targetType=target_type,
            targetDescription=target["description"],
            variables=json.dumps(
                [
                    {
                        "definition": v["definition"],
                        "symbol": v["symbol"],
                        "shape": v["shape"],
                    }
                    for v in state["variables"]
                ],
                indent=4,
            ),
            parameters=json.dumps(
                [
                    {
                        "definition": p["definition"],
                        "symbol": p["symbol"],
                        "shape": p["shape"],
                    }
                    for p in state["parameters"]
                ],
                indent=4,
            ),
        )

        cnt = 3
        while cnt > 0:
            cnt -= 1
            try:
                response = self.llm(prompt, max_length=1024, num_return_sequences=1)[0]['generated_text']
                print("=" * 10)
                print(response)
                print("=" * 10)
                output = response
                # delete until the first '```json'
                if "```json" in output:
                    output = output[output.find("```json") + 7 :]
                    output = output[: output.rfind("```")]

                # go back until the last character is a }
                while output[-1] != "}":
                    output = output[:-1]

                # go forward until the first character is a {
                while output[0] != "{":
                    output = output[1:]

                # if there are '$' in the output, remove them
                if "$" in output:
                    output = output.replace("$", "")

                # find "formulation": " in output
                formulation_start = output.find('"formulation"')
                # find "new_variables": " in output
                auxiliary_constraints_start = output.find('"auxiliary_constraints"')
                # go back until you find a closed bracket
                while output[auxiliary_constraints_start] != "}":
                    auxiliary_constraints_start -= 1
                while output[auxiliary_constraints_start] != '"':
                    auxiliary_constraints_start -= 1

                # extract the formulation
                formulation = output[
                    formulation_start + 16 : auxiliary_constraints_start
                ]
                # remove it from the output
                output = (
                    output[: formulation_start + 16]
                    + output[auxiliary_constraints_start:]
                )

                formulation = formulation.replace("\\\\", "\\")
                output = output.replace("\\", "\\\\")
                output = output.replace("\\", "\\\\")
                output = output.replace("\\\\quad", "\\\\")

                update = json.loads(output)
                update[target_type]["formulation"] = formulation

                # Extract related variables and parameters from the formulation
                related_stuff = self.get_related_stuff(
                    state, formulation, update["new_variables"]
                )
                update["variables_mentioned"] = related_stuff["variables_mentioned"]
                update["parameters_mentioned"] = related_stuff["parameters_mentioned"]

                if not "new_variables" in update.keys():
                    raise Exception("new_variables is not in the json file!")
                if not "formulation" in update[target_type].keys():
                    raise Exception("formulation is not in the json file!")
                if not "auxiliary_constraints" in update.keys():
                    update["auxiliary_constraints"] = []

                break
            except Exception as e:
                import traceback

                print(traceback.format_exc())
                print("=" * 10)
                print(e)
                print("=" * 10)
                print(prompt)
                print("=" * 10)
                print(response)
                print("=" * 10)
                print(
                    f"Invalid json format in {target_type} formulation!\n{e}\n Try again ..."
                )

        if cnt == 0:
            raise Exception("Invalid json format!")

        all_variable_symbols = [variable["symbol"] for variable in state["variables"]]

        for variable in update["new_variables"]:
            if variable["symbol"] in all_variable_symbols:
                # raise Exception(f"Variable {variable['symbol']} already exists!")
                continue
            else:
                variable["status"] = "formulated"
                state["variables"].append(variable)
                if not variable["symbol"] in update["variables_mentioned"]:
                    update["variables_mentioned"].append(variable["symbol"])

        target["formulation"] = update[target_type]["formulation"]
        target["status"] = "formulated"

        target["related_variables"] = update["variables_mentioned"]
        target["related_parameters"] = update["parameters_mentioned"]

        # Add auxiliary constraints
        if "auxiliary_constraints" in update.keys():
            for constraint in update["auxiliary_constraints"]:
                constraint["status"] = "formulated"

                constraint["formulation"] = (
                    constraint["formulation"]
                    .replace("\\\\", "\\")
                    .replace("\\\\", "\\")
                )

                # Extract related variables and parameters from the formulation
                related_stuff = self.get_related_stuff(
                    state, constraint["formulation"], []
                )
                constraint["related_variables"] = related_stuff["variables_mentioned"]
                constraint["related_parameters"] = related_stuff["parameters_mentioned"]

                state["constraint"].append(constraint)

        print(f"Formulation: {target['formulation']}")
        print("---")
        return

    def _fix_available_formulation(self, target, target_type: str, state):
        print(f"Fixing {target_type} ...")
        context = {}
        context[target_type] = {}
        context[target_type]["description"] = target["description"]

        context["variables"] = state["variables"]
        context["parameters"] = state["parameters"]
        context["formulation"] = target["formulation"]
        context["error"] = state["error_message"]

        prompt = fix_prompt_template.format(
            target=target_type,
            constraint=json.dumps(context[target_type]["description"], indent=4),
            variables=json.dumps(context["variables"], indent=4),
            parameters=json.dumps(context["parameters"], indent=4),
            formulation=json.dumps(context["formulation"], indent=4),
            json=json.dumps(target),
            error=context["error"],
        )

        cnt = 3
        while cnt > 0:
            cnt -= 1
            try:
                response = self.llm(prompt, max_length=1024, num_return_sequences=1)[0]['generated_text']
                # delete until the first '```json'
                output = response[response.find("```json") + 7 :]
                # delete until the last '```'
                output = output[: output.rfind("```")]

                output = output.replace(" \\", " \\\\")
                update = json.loads(output)

                break
            except Exception as e:
                print(e)
                print(
                    f"Invalid json format in {target_type} formulation! Try again ..."
                )

        if cnt == 0:
            raise Exception("Invalid json format!")

        target["formulation"] = update["formulation"]
        target["related_variables"] = update["related_variables"]
        target["related_parameters"] = update["related_parameters"]
        target["status"] = "formulated"

        return

    def generate_reply(self, task: str, state: Dict, sender: Agent) -> (str, Dict):
        # add some lines and characters around it to make the input interface nicer

        print("- Formulator agent is called!")
        print()

        self._formulate("constraint", state)
        self._formulate("objective", state)

        return "Formulation Done! Now we can write the code.", state

    def get_related_stuff(self, state, formulation, new_variables):
        ret = {}
        ret["variables_mentioned"] = []
        ret["parameters_mentioned"] = []

        # find all symbols enclosed in \textup{} in the formulation
        symbols_mentioned = []
        for i in range(len(formulation)):
            if formulation[i : i + 8] == "\\textup{":
                j = i + 8
                while formulation[j] != "}":
                    j += 1
                symbols_mentioned.append(formulation[i + 8 : j])

        all_parameter_symbols = [
            parameter["symbol"] for parameter in state["parameters"]
        ]
        all_variable_symbols = [variable["symbol"] for variable in state["variables"]]
        all_variable_symbols += [variable["symbol"] for variable in new_variables]

        # print(all_parameter_symbols)
        # print(all_variable_symbols)

        for symbol in symbols_mentioned:
            if symbol in all_parameter_symbols:
                ret["parameters_mentioned"].append(symbol)
            elif symbol in all_variable_symbols:
                ret["variables_mentioned"].append(symbol)
            elif symbol.lower().strip() in [
                "min",
                "max",
                "subject to",
                "s.t.",
                "st",
                "minimize",
                "maximize",
                "sum",
                "for all",
                "forall",
                "such that",
                "and",
                "or",
                "if",
                "then",
                "else",
                "otherwise",
                "for each",
                "exists",
                "foreach",
            ]:
                continue
            elif " " in symbol:
                continue
            else:
                raise Exception(f"Symbol {symbol} is not defined!")

        return ret

In [21]:
# FORMULATOR EXAMPLE
from typing import Dict

import json
import openai

fix_prompt_template = """
You are a mathematical formulator working with a team of optimization experts. The objective is to tackle a complex optimization problem, and your role is to fix a previously modelled {target}.

Recall that the {target} you modelled was

-----
{constraint}
-----

and your formulation you provided was

-----
{formulation}
-----

The error message is

-----
{error}
-----

Here are the variables you have so far defined:

-----
{variables}
-----

Here are the parameters of the problem

-----
{parameters}
-----

Your task is carefully inspect the old {target} and fix it when you find it actually wrong.
After fixing it modify the formulation. Please return the fixed JSON string for the formulation.

The current JSON is

-----
{json}
-----

"""

prompt_template = """
You are an expert mathematical formulator and an optimization professor at a top university. Your task is to model {targetType} of the problem in the standard LP or MILP form.

Here is a {targetType} we need you to model:
-----
{targetDescription}
-----

Here is some context on the problem:
-----
{background}
-----

Here is the list of available variables:
-----
{variables}
-----

And finally, here is list of input parameters:
-----
{parameters}
-----

First, take a deep breath and explain how we should define the {targetType}. Feel free to define new variables if you think it is necessary. Then, generate a json file accordingly with the following format (STICK TO THIS FORMAT!):


{{
    "{targetType}": {{
      "description": "The description of the {targetType}",
      "formulation": "The LaTeX mathematical expression representing the formulation of the {targetType}"
    }},
    "auxiliary_constraints": [
        {{
            "description": "The description of the auxiliary constraint",
            "formulation": "The LaTeX mathematical expression representing the formulation of the auxiliary constraint"
        }}
    ]
    "new_variables": [
        {{
            "definition": "The definition of the variable",
            "symbol": "The symbol for the variable",
            "shape": [ "symbol1", "symbol2", ... ]
        }}
    ],

}}

- Your formulation should be in LaTeX mathematical format (do not include the $ symbols).
- Note that I'm going to use python json.loads() function to parse the json file, so please make sure the format is correct (don't add ',' before enclosing '}}' or ']' characters.
- Generate the complete json file and don't omit anything.
- Use '```json' and '```' to enclose the json file.
- Important: You can not define new parameters. You can only define new variables.Use CamelCase and full words for new variable symbols, and do not include indices in the symbol (e.g. ItemsSold instead of itemsSold or items_sold or ItemsSold_i)
- Use \\textup{{}} when writing variable and parameter names. For example (\\sum_{{i=1}}^{{N}} \\textup{{ItemsSold}}_{{i}} instead of \\sum_{{i=1}}^{{N}} ItemsSold_{{i}})
- Use \\quad for spaces.
- Use empty list ([]) if no new variables are defined.
- Always use non-strict inequalities (e.g. \\leq instead of <), even if the constraint is strict.
- Define auxiliary constraints when necessary. Set it to an empty list ([]) if no auxiliary constraints are needed. If new auxiliary constraints need new variables, add them to the "new_variables" list too.

Take a deep breath and solve the problem step by step.

"""


class Formulator(Agent):
    def __init__(self, client: openai.Client, **kwargs):
        super().__init__(
            name="Formulator",
            description="This is a mathematical formulator agent that is an expert in mathematical and optimization modeling and can define and modify variables, constraints, and objective functions. Does 3 things: 1) Defining constraints, variables, and objective functions, 2) Modifying constraints, variables, and objective functions, 3) Other things related to mathematical formulation. If the history is empty, start from this agent.",
            client=client,
            **kwargs,
        )

    def _formulate(self, target_type: str, state):
        for target in state[target_type]:
            if target["status"] == "not_formulated":
                self._create_new_formulation(target, target_type, state)
            elif target["status"] == "runtime_error":
                self._fix_available_formulation(target, target_type, state)
            elif target["status"] == "formulated":
                continue
            else:
                error_msg = f"Invalid status: {json.dumps(target, indent=4)}"
                raise RuntimeError(error_msg)

        return

    def _create_new_formulation(self, target, target_type: str, state):
        print(f"Formulating {target_type} ...")
        context = {}
        context[target_type] = {}
        prompt = prompt_template.format(
            background=state["background"],
            targetType=target_type,
            targetDescription=target["description"],
            variables=json.dumps(
                [
                    {
                        "definition": v["definition"],
                        "symbol": v["symbol"],
                        "shape": v["shape"],
                    }
                    for v in state["variables"]
                ],
                indent=4,
            ),
            parameters=json.dumps(
                [
                    {
                        "definition": p["definition"],
                        "symbol": p["symbol"],
                        "shape": p["shape"],
                    }
                    for p in state["parameters"]
                ],
                indent=4,
            ),
        )

        cnt = 3
        while cnt > 0:
            cnt -= 1
            try:
                response = self.llm_call(prompt=prompt, seed=cnt)
                print("=" * 10)
                print(response)
                print("=" * 10)
                output = response
                # delete until the first '```json'
                if "```json" in output:
                    output = output[output.find("```json") + 7 :]
                    output = output[: output.rfind("```")]

                # go back until the last character is a }
                while output[-1] != "}":
                    output = output[:-1]

                # go forward until the first character is a {
                while output[0] != "{":
                    output = output[1:]

                # if there are '$' in the output, remove them
                if "$" in output:
                    output = output.replace("$", "")

                # find "formulation": " in output
                formulation_start = output.find('"formulation"')
                # find "new_variables": " in output
                auxiliary_constraints_start = output.find('"auxiliary_constraints"')
                # go back until you find a closed bracket
                while output[auxiliary_constraints_start] != "}":
                    auxiliary_constraints_start -= 1
                while output[auxiliary_constraints_start] != '"':
                    auxiliary_constraints_start -= 1

                # extract the formulation
                formulation = output[
                    formulation_start + 16 : auxiliary_constraints_start
                ]
                # remove it from the output
                output = (
                    output[: formulation_start + 16]
                    + output[auxiliary_constraints_start:]
                )

                formulation = formulation.replace("\\\\", "\\")
                output = output.replace("\\", "\\\\")
                output = output.replace("\\", "\\\\")
                output = output.replace("\\\\quad", "\\\\")

                update = json.loads(output)
                update[target_type]["formulation"] = formulation

                # Extract related variables and parameters from the formulation
                related_stuff = self.get_related_stuff(
                    state, formulation, update["new_variables"]
                )
                update["variables_mentioned"] = related_stuff["variables_mentioned"]
                update["parameters_mentioned"] = related_stuff["parameters_mentioned"]

                if not "new_variables" in update.keys():
                    raise Exception("new_variables is not in the json file!")
                if not "formulation" in update[target_type].keys():
                    raise Exception("formulation is not in the json file!")
                if not "auxiliary_constraints" in update.keys():
                    update["auxiliary_constraints"] = []

                break
            except Exception as e:
                import traceback

                print(traceback.format_exc())
                print("=" * 10)
                print(e)
                print("=" * 10)
                print(prompt)
                print("=" * 10)
                print(response)
                print("=" * 10)
                print(
                    f"Invalid json format in {target_type} formulation!\n{e}\n Try again ..."
                )

        if cnt == 0:
            raise Exception("Invalid json format!")

        all_variable_symbols = [variable["symbol"] for variable in state["variables"]]

        for variable in update["new_variables"]:
            if variable["symbol"] in all_variable_symbols:
                # raise Exception(f"Variable {variable['symbol']} already exists!")
                continue
            else:
                variable["status"] = "formulated"
                state["variables"].append(variable)
                if not variable["symbol"] in update["variables_mentioned"]:
                    update["variables_mentioned"].append(variable["symbol"])

        target["formulation"] = update[target_type]["formulation"]
        target["status"] = "formulated"

        target["related_variables"] = update["variables_mentioned"]
        target["related_parameters"] = update["parameters_mentioned"]

        # Add auxiliary constraints
        if "auxiliary_constraints" in update.keys():
            for constraint in update["auxiliary_constraints"]:
                constraint["status"] = "formulated"

                constraint["formulation"] = (
                    constraint["formulation"]
                    .replace("\\\\", "\\")
                    .replace("\\\\", "\\")
                )

                # Extract related variables and parameters from the formulation
                related_stuff = self.get_related_stuff(
                    state, constraint["formulation"], []
                )
                constraint["related_variables"] = related_stuff["variables_mentioned"]
                constraint["related_parameters"] = related_stuff["parameters_mentioned"]

                state["constraint"].append(constraint)

        print(f"Formulation: {target['formulation']}")
        print("---")
        return

    def _fix_available_formulation(self, target, target_type: str, state):
        print(f"Fixing {target_type} ...")
        context = {}
        context[target_type] = {}
        context[target_type]["description"] = target["description"]

        context["variables"] = state["variables"]
        context["parameters"] = state["parameters"]
        context["formulation"] = target["formulation"]
        context["error"] = state["error_message"]

        prompt = fix_prompt_template.format(
            target=target_type,
            constraint=json.dumps(context[target_type]["description"], indent=4),
            variables=json.dumps(context["variables"], indent=4),
            parameters=json.dumps(context["parameters"], indent=4),
            formulation=json.dumps(context["formulation"], indent=4),
            json=json.dumps(target),
            error=context["error"],
        )

        cnt = 3
        while cnt > 0:
            cnt -= 1
            try:
                output = self.llm_call(prompt=prompt)
                # delete until the first '```json'
                output = output[output.find("```json") + 7 :]
                # delete until the last '```'
                output = output[: output.rfind("```")]

                output = output.replace(" \\", " \\\\")
                update = json.loads(output)

                break
            except Exception as e:
                print(e)
                print(
                    f"Invalid json format in {target_type} formulation! Try again ..."
                )

        if cnt == 0:
            raise Exception("Invalid json format!")

        target["formulation"] = update["formulation"]
        target["related_variables"] = update["related_variables"]
        target["related_parameters"] = update["related_parameters"]
        target["status"] = "formulated"

        return

    def generate_reply(self, task: str, state: Dict, sender: Agent) -> (str, Dict):
        # add some lines and characters around it to make the input interface nicer

        print("- Formulator agent is called!")
        print()

        self._formulate("constraint", state)
        self._formulate("objective", state)

        return "Formulation Done! Now we can write the code.", state

    def get_related_stuff(self, state, formulation, new_variables):
        ret = {}
        ret["variables_mentioned"] = []
        ret["parameters_mentioned"] = []

        # find all symbols enclosed in \textup{} in the formulation
        symbols_mentioned = []
        for i in range(len(formulation)):
            if formulation[i : i + 8] == "\\textup{":
                j = i + 8
                while formulation[j] != "}":
                    j += 1
                symbols_mentioned.append(formulation[i + 8 : j])

        all_parameter_symbols = [
            parameter["symbol"] for parameter in state["parameters"]
        ]
        all_variable_symbols = [variable["symbol"] for variable in state["variables"]]
        all_variable_symbols += [variable["symbol"] for variable in new_variables]

        # print(all_parameter_symbols)
        # print(all_variable_symbols)

        for symbol in symbols_mentioned:
            if symbol in all_parameter_symbols:
                ret["parameters_mentioned"].append(symbol)
            elif symbol in all_variable_symbols:
                ret["variables_mentioned"].append(symbol)
            elif symbol.lower().strip() in [
                "min",
                "max",
                "subject to",
                "s.t.",
                "st",
                "minimize",
                "maximize",
                "sum",
                "for all",
                "forall",
                "such that",
                "and",
                "or",
                "if",
                "then",
                "else",
                "otherwise",
                "for each",
                "exists",
                "foreach",
            ]:
                continue
            elif " " in symbol:
                continue
            else:
                raise Exception(f"Symbol {symbol} is not defined!")

        return ret

In [31]:
# Install necessary libraries
!pip install openai transformers

# Import necessary modules
from typing import Dict
import json
import openai
from transformers import pipeline

# Define the Agent class (from agent.py)
class Agent:
    def __init__(self, name, description, client, llm="gpt-3.5-turbo", **kwargs):
        self.name = name
        self.description = description
        self.client = client
        self.system_prompt = "You're a helpful assistant."
        self.kwargs = kwargs
        self.llm = llm

    def llm_call(
        self,
        prompt: Optional[str] = None,
        messages: Optional[List] = None,
        seed: int = 10,
    ) -> str:
        model = self.llm
        # make sure exactly one of prompt or messages is provided
        assert (prompt is None) != (messages is None)
        # make sure if messages is provided, it is a list of dicts with role and content
        if messages is not None:
            assert isinstance(messages, list)
            for message in messages:
                assert isinstance(message, dict)
                assert "role" in message
                assert "content" in message

        if not prompt is None:
            messages = [
                {"role": "system", "content": self.system_prompt},
                {"role": "user", "content": prompt},
            ]

        if type(self.client) in [OpenAI, Client]:
            completion = self.client.chat.completions.create(
                model=model,
                messages=messages,
                seed=seed,
            )
            content = completion.choices[0].message.content

        elif type(self.client) == MistralClient:
            messages = [
                ChatMessage(role=message["role"], content=message["content"])
                for message in messages
            ]
            completion = self.client.chat(
                model=model,
                messages=messages,
            )
            content = completion.choices[0].message.content

        return content

    def generate_reply(
        self,
        task: str,
        state: Dict,
        sender: "Agent",
    ) -> Tuple[str, Dict]:
        return (
            "This is a reply from the agent. REPLY NOT IMPLEMENTED! Terminate the whole process!",
            state,
        )

# Define the Formulator class (from formulator.py)
fix_prompt_template = """
You are a mathematical formulator working with a team of optimization experts. The objective is to tackle a complex optimization problem, and your role is to fix a previously modelled {target}.

Recall that the {target} you modelled was

-----
{constraint}
-----

and your formulation you provided was

-----
{formulation}
-----

The error message is

-----
{error}
-----

Here are the variables you have so far defined:

-----
{variables}
-----

Here are the parameters of the problem

-----
{parameters}
-----

Your task is carefully inspect the old {target} and fix it when you find it actually wrong.
After fixing it modify the formulation. Please return the fixed JSON string for the formulation.

The current JSON is

-----
{json}
-----

"""

prompt_template = """
You are an expert mathematical formulator and an optimization professor at a top university. Your task is to model {targetType} of the problem in the standard LP or MILP form.

Here is a {targetType} we need you to model:
-----
{targetDescription}
-----

Here is some context on the problem:
-----
{background}
-----

Here is the list of available variables:
-----
{variables}
-----

And finally, here is list of input parameters:
-----
{parameters}
-----

First, take a deep breath and explain how we should define the {targetType}. Feel free to define new variables if you think it is necessary. Then, generate a json file accordingly with the following format (STICK TO THIS FORMAT!):


{{
    "{targetType}": {{
      "description": "The description of the {targetType}",
      "formulation": "The LaTeX mathematical expression representing the formulation of the {targetType}"
    }},
    "auxiliary_constraints": [
        {{
            "description": "The description of the auxiliary constraint",
            "formulation": "The LaTeX mathematical expression representing the formulation of the auxiliary constraint"
        }}
    ]
    "new_variables": [
        {{
            "definition": "The definition of the variable",
            "symbol": "The symbol for the variable",
            "shape": [ "symbol1", "symbol2", ... ]
        }}
    ],

}}

- Your formulation should be in LaTeX mathematical format (do not include the $ symbols).
- Note that I'm going to use python json.loads() function to parse the json file, so please make sure the format is correct (don't add ',' before enclosing '}}' or ']' characters.
- Generate the complete json file and don't omit anything.
- Use '```json' and '```' to enclose the json file.
- Important: You can not define new parameters. You can only define new variables.Use CamelCase and full words for new variable symbols, and do not include indices in the symbol (e.g. ItemsSold instead of itemsSold or items_sold or ItemsSold_i)
- Use \\textup{{}} when writing variable and parameter names. For example (\\sum_{{i=1}}^{{N}} \\textup{{ItemsSold}}_{{i}} instead of \\sum_{{i=1}}^{{N}} ItemsSold_{{i}})
- Use \\quad for spaces.
- Use empty list ([]) if no new variables are defined.
- Always use non-strict inequalities (e.g. \\leq instead of <), even if the constraint is strict.
- Define auxiliary constraints when necessary. Set it to an empty list ([]) if no auxiliary constraints are needed. If new auxiliary constraints need new variables, add them to the "new_variables" list too.

Take a deep breath and solve the problem step by step.

"""

class Formulator(Agent):
    def __init__(self, client=None, **kwargs):
        super().__init__(
            name="Formulator",
            description="This is a mathematical formulator agent that is an expert in mathematical and optimization modeling and can define and modify variables, constraints, and objective functions. Does 3 things: 1) Defining constraints, variables, and objective functions, 2) Modifying constraints, variables, and objective functions, 3) Other things related to mathematical formulation. If the history is empty, start from this agent.",
            client=client,
            **kwargs,
        )
        self.llm = pipeline('text-generation', model='gpt2')

    def _formulate(self, target_type: str, state):
        for target in state[target_type]:
            if target["status"] == "not_formulated":
                self._create_new_formulation(target, target_type, state)
            elif target["status"] == "runtime_error":
                self._fix_available_formulation(target, target_type, state)
            elif target["status"] == "formulated":
                continue
            else:
                error_msg = f"Invalid status: {json.dumps(target, indent=4)}"
                raise RuntimeError(error_msg)

        return

    def _create_new_formulation(self, target, target_type: str, state):
        print(f"Formulating {target_type} ...")
        context = {}
        context[target_type] = {}
        prompt = prompt_template.format(
            background=state["background"],
            targetType=target_type,
            targetDescription=target["description"],
            variables=json.dumps(
                [
                    {
                        "definition": v["definition"],
                        "symbol": v["symbol"],
                        "shape": v["shape"],
                    }
                    for v in state["variables"]
                ],
                indent=4,
            ),
            parameters=json.dumps(
                [
                    {
                        "definition": p["definition"],
                        "symbol": p["symbol"],
                        "shape": p["shape"],
                    }
                    for p in state["parameters"]
                ],
                indent=4,
            ),
        )

        cnt = 3
        while cnt > 0:
            cnt -= 1
            try:
                response = self.llm(prompt, max_length=1024, num_return_sequences=1)[0]['generated_text']
                print("=" * 10)
                print(response)
                print("=" * 10)
                output = response
                # delete until the first '```json'
                if "```json" in output:
                    output = output[output.find("```json") + 7 :]
                    output = output[: output.rfind("```")]

                # go back until the last character is a }
                while output[-1] != "}":
                    output = output[:-1]

                # go forward until the first character is a {
                while output[0] != "{":
                    output = output[1:]

                # if there are '$' in the output, remove them
                if "$" in output:
                    output = output.replace("$", "")

                # find "formulation": " in output
                formulation_start = output.find('"formulation"')
                # find "new_variables": " in output
                auxiliary_constraints_start = output.find('"auxiliary_constraints"')
                # go back until you find a closed bracket
                while output[auxiliary_constraints_start] != "}":
                    auxiliary_constraints_start -= 1
                while output[auxiliary_constraints_start] != '"':
                    auxiliary_constraints_start -= 1

                # extract the formulation
                formulation = output[
                    formulation_start + 16 : auxiliary_constraints_start
                ]
                # remove it from the output
                output = (
                    output[: formulation_start + 16]
                    + output[auxiliary_constraints_start:]
                )

                formulation = formulation.replace("\\\\", "\\")
                output = output.replace("\\", "\\\\")
                output = output.replace("\\", "\\\\")
                output = output.replace("\\\\quad", "\\\\")

                update = json.loads(output)
                update[target_type]["formulation"] = formulation

                # Extract related variables and parameters from the formulation
                related_stuff = self.get_related_stuff(
                    state, formulation, update["new_variables"]
                )
                update["variables_mentioned"] = related_stuff["variables_mentioned"]
                update["parameters_mentioned"] = related_stuff["parameters_mentioned"]

                if not "new_variables" in update.keys():
                    raise Exception("new_variables is not in the json file!")
                if not "formulation" in update[target_type].keys():
                    raise Exception("formulation is not in the json file!")
                if not "auxiliary_constraints" in update.keys():
                    update["auxiliary_constraints"] = []

                break
            except Exception as e:
                import traceback

                print(traceback.format_exc())
                print("=" * 10)
                print(e)
                print("=" * 10)
                print(prompt)
                print("=" * 10)
                print(response)
                print("=" * 10)
                print(
                    f"Invalid json format in {target_type} formulation!\n{e}\n Try again ..."
                )

        if cnt == 0:
            raise Exception("Invalid json format!")

        all_variable_symbols = [variable["symbol"] for variable in state["variables"]]

        for variable in update["new_variables"]:
            if variable["symbol"] in all_variable_symbols:
                # raise Exception(f"Variable {variable['symbol']} already exists!")
                continue
            else:
                variable["status"] = "formulated"
                state["variables"].append(variable)
                if not variable["symbol"] in update["variables_mentioned"]:
                    update["variables_mentioned"].append(variable["symbol"])

        target["formulation"] = update[target_type]["formulation"]
        target["status"] = "formulated"

        target["related_variables"] = update["variables_mentioned"]
        target["related_parameters"] = update["parameters_mentioned"]

        # Add auxiliary constraints
        if "auxiliary_constraints" in update.keys():
            for constraint in update["auxiliary_constraints"]:
                constraint["status"] = "formulated"

                constraint["formulation"] = (
                    constraint["formulation"]
                    .replace("\\\\", "\\")
                    .replace("\\\\", "\\")
                )

                # Extract related variables and parameters from the formulation
                related_stuff = self.get_related_stuff(
                    state, constraint["formulation"], []
                )
                constraint["related_variables"] = related_stuff["variables_mentioned"]
                constraint["related_parameters"] = related_stuff["parameters_mentioned"]

                state["constraint"].append(constraint)

        print(f"Formulation: {target['formulation']}")
        print("---")
        return

    def _fix_available_formulation(self, target, target_type: str, state):
        print(f"Fixing {target_type} ...")
        context = {}
        context[target_type] = {}
        context[target_type]["description"] = target["description"]

        context["variables"] = state["variables"]
        context["parameters"] = state["parameters"]
        context["formulation"] = target["formulation"]
        context["error"] = state["error_message"]

        prompt = fix_prompt_template.format(
            target=target_type,
            constraint=json.dumps(context[target_type]["description"], indent=4),
            variables=json.dumps(context["variables"], indent=4),
            parameters=json.dumps(context["parameters"], indent=4),
            formulation=json.dumps(context["formulation"], indent=4),
            json=json.dumps(target),
            error=context["error"],
        )

        cnt = 3
        while cnt > 0:
            cnt -= 1
            try:
                response = self.llm(prompt, max_length=1024, num_return_sequences=1)[0]['generated_text']
                # delete until the first '```json'
                output = response[response.find("```json") + 7 :]
                # delete until the last '```'
                output = output[: output.rfind("```")]

                output = output.replace(" \\", " \\\\")
                update = json.loads(output)

                break
            except Exception as e:
                print(e)
                print(
                    f"Invalid json format in {target_type} formulation! Try again ..."
                )

        if cnt == 0:
            raise Exception("Invalid json format!")

        target["formulation"] = update["formulation"]
        target["related_variables"] = update["related_variables"]
        target["related_parameters"] = update["related_parameters"]
        target["status"] = "formulated"

        return

    def generate_reply(self, task: str, state: Dict, sender: Agent) -> Tuple[str, Dict]:
        # add some lines and characters around it to make the input interface nicer

        print("- Formulator agent is called!")
        print()

        self._formulate("constraint", state)
        self._formulate("objective", state)

        return "Formulation Done! Now we can write the code.", state

    def get_related_stuff(self, state, formulation, new_variables):
        ret = {}
        ret["variables_mentioned"] = []
        ret["parameters_mentioned"] = []

        # find all symbols enclosed in \textup{} in the formulation
        symbols_mentioned = []
        for i in range(len(formulation)):
            if formulation[i : i + 8] == "\\textup{":
                j = i + 8
                while formulation[j] != "}":
                    j += 1
                symbols_mentioned.append(formulation[i + 8 : j])

        all_parameter_symbols = [
            parameter["symbol"] for parameter in state["parameters"]
        ]
        all_variable_symbols = [variable["symbol"] for variable in state["variables"]]
        all_variable_symbols += [variable["symbol"] for variable in new_variables]

        # print(all_parameter_symbols)
        # print(all_variable_symbols)

        for symbol in symbols_mentioned:
            if symbol in all_parameter_symbols:
                ret["parameters_mentioned"].append(symbol)
            elif symbol in all_variable_symbols:
                ret["variables_mentioned"].append(symbol)
            elif symbol.lower().strip() in [
                "min",
                "max",
                "subject to",
                "s.t.",
                "st",
                "minimize",
                "maximize",
                "sum",
                "for all",
                "forall",
                "such that",
                "and",
                "or",
                "if",
                "then",
                "else",
                "otherwise",
                "for each",
                "exists",
                "foreach",
            ]:
                continue
            elif " " in symbol:
                continue
            else:
                raise Exception(f"Symbol {symbol} is not defined!")

        return ret

# Define the GroupChatManager class (from manager.py)
class GroupChatManager(Agent):
    def __init__(
        self, client: openai.Client, agents: List[Agent], max_rounds: int = 12, **kwargs
    ):
        super().__init__(
            name="GroupChatManager",
            description="This is a manager agent that chooses which agent to work on the problem next and organizes the conversation within its team.",
            client=client,
            **kwargs,
        )

        self.agents = agents
        self.conversation_state = {
            "round": 0,
        }
        self.max_rounds = max_rounds
        self.history = []
        self.prompt_template = """

You're a manager in a team of optimization experts. The goal of the team is to solve an optimization problem. Your task is to choose the next expert to work on the problem based on the current situation.
- The user has already given us the problem description, the objective function, and the parameters. Only call the user proxy if there is a problem or something ambiguous or missing.

Here's the list of agents in your team:
-----
{agents}
-----

And here's the history of the conversation so far:
-----
{history}
-----


Considering the history, if you think the problem is solved, type DONE. Otherwise, generate a json file with the following format:
{{
    "agent_name": "Name of the agent you want to call next",
    "task": "The task you want the agent to carry out"
}}

to identify the next agent to work on the problem, and also the task it has to carry out.
- If there is a runtime error, ask the the prorammer agent to fix it.
- Only generate the json file, and don't generate any other text.
- If the latest message in history says that the code is fixed, ask the evaluator agent to evaluate the code!

"""

    def solve(self, state: Dict) -> Tuple[str, Dict]:
      self.history = []

      while True:
        if self.conversation_state["round"] >= self.max_rounds:
            return "The problem is not solved.", state

        print("=" * 20)
        print("=" * 20)
        print("Round", self.conversation_state["round"])
        # print(json.dumps(state, indent=4))

        agents_list = "".join(
            [
                "-" + agent.name + ": " + agent.description + "\n"
                for agent in self.agents
            ]
        )

        prompt = self.prompt_template.format(
            agents=agents_list,
            history="\n".join([json.dumps(item[0]) for item in self.history]),
        )

        cnt = 3
        while True and cnt > 0:
            try:
                response = self.llm_call(prompt=prompt, seed=cnt)

                decision = response.strip()
                if "```json" in decision:
                    decision = decision.split("```json")[1].split("```")[0]
                decision = decision.replace("\\", "")

                if decision == "DONE":
                    print("DONE")
                    return "The problem is solved.", state
                decision = json.loads(decision)
                break

            except Exception as e:
                print(response)
                print(e)
                cnt -= 1

                print("Invalid decision. Trying again ...")
                if cnt == 0:
                    import traceback

                    err = traceback.format_exc()
                    print(err)

        print(
            "---- History:\n",
            "\n".join([json.dumps(item[0]) for item in self.history]),
        )

        print(f"\n---- Decision:||{decision}||\n")

        # wait for the user to press enter
        # input()

        if not decision["agent_name"] in [agent.name for agent in self.agents]:
            raise ValueError(
                f"Decision {decision} is not a valid agent name. Please choose from {self.agents}"
            )
        else:
            agent = [
                agent
                for agent in self.agents
                if agent.name == decision["agent_name"]
            ][0]

            message, new_state = agent.generate_reply(
                task=decision["task"],
                state=state,
                sender=self,
            )

            with open(
                f"{state['log_folder']}/log_{self.conversation_state['round']}.json",
                "w",
            ) as f:
                json.dump(state, f, indent=4)

            state = new_state

            decision["result"] = message
            self.history.append((decision, state))

            with open(state["log_folder"] + "/selection_log.json", "w") as f:
                json.dump([d for (d, s) in self.history], f, indent=4)

            if "code" in state:
                with open(state["log_folder"] + "/code.py", "w") as f:
                    f.write(state["code"])

            self.conversation_state["round"] += 1